# 🧠 Financial Sentiment Model Explainability Dashboard

## Overview
This notebook provides comprehensive explainability analysis for the fine-tuned TinyBERT financial sentiment classification model. It includes four complementary explanation methods accessible through an interactive dashboard.

### Explanation Methods
- **🎯 SHAP**: Game-theory based feature importance
- **🔍 LIME**: Local interpretable model-agnostic explanations 
- **👁️ Attention**: Model attention head visualization
- **🌡️ GradCAM**: Gradient-based visual attribution

### Dashboard Features
- **Mistake Analysis**: Examine specific model errors
- **Custom Text Analysis**: Test any financial text
- **Interactive Interface**: Tabbed layout for easy comparison
- **On-demand Computation**: Optimized performance

## 1. 📦 Setup & Imports

In [1]:
import os
os.chdir('/Users/matthew/Documents/deepmind_internship')
print(f"Working directory: {os.getcwd()}")

Working directory: /Users/matthew/Documents/deepmind_internship


In [2]:
# Core libraries
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Model and tokenizer
from transformers import BertTokenizerFast, BertForSequenceClassification
from sklearn.preprocessing import LabelEncoder

# Explainability libraries
import shap
from lime.lime_text import LimeTextExplainer
from bertviz import head_view
from captum.attr import LayerGradCam

# Dashboard components
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

## 2. ⚙️ Data Loading

**Simple Data Loading** - Using the exact working approach from the other notebook:

- **Fixed Configuration**: Uses FinancialPhraseBank dataset with known structure
- **Proven Encoding**: latin-1 encoding (works reliably)
- **Standard Columns**: 'label' and 'sentence' columns with header=None
- **Train-Test Split**: Consistent 25% test split with stratification
- **Quote Handling**: Automatic removal of extra quotes from text

In [3]:
# Load the fine-tuned model
MODEL_DIR = Path('models/tinybert-financial-classifier')
DATA_FILE = 'data/FinancialPhraseBank/all-data.csv'
RANDOM_SEED = 42
TEST_SIZE = 0.25

model = BertForSequenceClassification.from_pretrained(MODEL_DIR)
tokenizer = BertTokenizerFast.from_pretrained(MODEL_DIR)

# Load label encoder
import pickle
with open(MODEL_DIR / 'label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

# Load data using the EXACT working approach from the other notebook
from sklearn.model_selection import train_test_split

# Load data with correct encoding and column names (matching training notebook)
df = pd.read_csv(DATA_FILE, header=None, names=["label", "sentence"], encoding="latin-1")
df["sentence"] = df["sentence"].str.strip('"')  # Remove extra quotes

# Create train-test split with same parameters as training
train_df, test_df = train_test_split(
    df, 
    test_size=TEST_SIZE, 
    random_state=RANDOM_SEED, 
    stratify=df['label']
)

# Extract test data
test_texts = test_df['sentence'].tolist()[:1000]  # Limit to 1000 for demo
test_labels = label_encoder.transform(test_df['label'].tolist()[:1000])

print(f"✅ Model loaded: {MODEL_DIR}")
print(f"📊 Full dataset: {len(df)} samples")
print(f"📊 Test samples: {len(test_texts)}")
print(f"🏷️ Labels: {list(label_encoder.classes_)}")
print(f"📈 Label distribution: {dict(zip(*np.unique(test_labels, return_counts=True)))}")
print(f"📝 Sample text: {test_texts[0][:100]}...")
print("✅ Data loaded successfully with correct encoding")

✅ Model loaded: models/tinybert-financial-classifier
📊 Full dataset: 4846 samples
📊 Test samples: 1000
🏷️ Labels: ['negative', 'neutral', 'positive']
📈 Label distribution: {0: 129, 1: 601, 2: 270}
📝 Sample text: Le Lay succeeds Walter G++nter and will be based in Finland ....
✅ Data loaded successfully with correct encoding


## 3. 🔮 Model Predictions

In [4]:
# Get model predictions
model.eval()
predictions = []
confidences = []

with torch.no_grad():
    for text in test_texts:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=-1)
        pred = torch.argmax(probs, dim=-1).item()
        conf = torch.max(probs).item()
        
        predictions.append(pred)
        confidences.append(conf)

predictions = np.array(predictions)
confidences = np.array(confidences)

# Calculate accuracy
accuracy = (predictions == test_labels).mean()
print(f"🎯 Accuracy: {accuracy:.3f}")
print(f"❌ Misclassifications: {(predictions != test_labels).sum()}")

🎯 Accuracy: 0.791
❌ Misclassifications: 209


## 4. 🎛️ Explainability Dashboard

In [ ]:
class ExplainabilityDashboard:
    def __init__(self, model, tokenizer, test_texts, test_labels, predictions, label_encoder):
        self.model = model
        self.tokenizer = tokenizer
        self.test_texts = test_texts
        self.test_labels = test_labels
        self.predictions = predictions
        self.label_encoder = label_encoder
        self.setup_dashboard()
    
    def setup_dashboard(self):
        # Input widgets
        self.text_input = widgets.Textarea(
            value="The company reported strong quarterly earnings with revenue growth of 15%.",
            placeholder="Enter financial text to analyze...",
            description="Text:",
            layout=widgets.Layout(width='100%', height='80px')
        )
        
        # Get misclassified examples for dropdown
        misclassified_indices = [i for i in range(len(self.test_texts)) 
                               if self.predictions[i] != self.test_labels[i]]
        
        self.mistake_dropdown = widgets.Dropdown(
            options=[(f"Mistake {i+1}: {self.test_texts[idx][:50]}...", idx) 
                    for i, idx in enumerate(misclassified_indices[:20])],
            description="Analyze Mistake:"
        )
        
        self.analyze_button = widgets.Button(
            description="🔍 Analyze Text",
            button_style='primary'
        )
        
        self.mistake_button = widgets.Button(
            description="🔍 Analyze Mistake",
            button_style='warning'
        )
        
        # Output area
        self.output = widgets.Output()
        
        # Event handlers
        self.analyze_button.on_click(self.analyze_custom_text)
        self.mistake_button.on_click(self.analyze_mistake)
        
        # Layout
        self.dashboard = widgets.VBox([
            widgets.HTML("<h3>🧠 Explainability Dashboard</h3>"),
            widgets.HBox([self.text_input]),
            widgets.HBox([self.analyze_button]),
            widgets.HTML("<hr><h4>Or analyze a model mistake:</h4>"),
            widgets.HBox([self.mistake_dropdown, self.mistake_button]),
            self.output
        ])
    
    def analyze_custom_text(self, _):
        if not self.text_input.value.strip():
            return
        
        with self.output:
            clear_output()
            self.run_analysis(self.text_input.value)
    
    def analyze_mistake(self, _):
        idx = self.mistake_dropdown.value
        text = self.test_texts[idx]
        true_label = self.label_encoder.inverse_transform([self.test_labels[idx]])[0]
        pred_label = self.label_encoder.inverse_transform([self.predictions[idx]])[0]
        
        with self.output:
            clear_output()
            print(f"🔍 Analyzing Mistake: True={true_label}, Predicted={pred_label}")
            print("-" * 60)
            self.run_analysis(text)
    
    def run_analysis(self, text):
        # Get prediction
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
        outputs = self.model(**inputs)
        probs = torch.softmax(outputs.logits, dim=-1)
        pred_idx = torch.argmax(probs, dim=-1).item()
        confidence = torch.max(probs).item()
        pred_label = self.label_encoder.inverse_transform([pred_idx])[0]
        
        print(f"📊 Prediction: {pred_label} (confidence: {confidence:.3f})")
        print(f"📝 Text: {text}")
        print("\n" + "="*80)
        
        # Run all explanation methods
        self.run_shap_analysis(text)
        self.run_lime_analysis(text)
        self.run_attention_analysis(text)
        self.run_gradcam_analysis(text)
        print("\n" + "="*80)
        print("✅ Analysis complete!")
    
    def run_shap_analysis(self, text):
        """SHAP explainability analysis"""
        print("\n🎯 SHAP Analysis:")
        try:
            # Create SHAP explainer for transformers
            def model_wrapper(texts):
                """Wrapper function for SHAP"""
                if isinstance(texts, str):
                    texts = [texts]
                
                predictions = []
                for t in texts:
                    if not t.strip():
                        predictions.append([0.33, 0.33, 0.34])
                        continue
                    
                    inputs = self.tokenizer(t, return_tensors="pt", truncation=True, 
                                          padding=True, max_length=128)
                    with torch.no_grad():
                        outputs = self.model(**inputs)
                        probs = torch.softmax(outputs.logits, dim=-1).cpu().numpy()[0]
                    predictions.append(probs)
                
                return np.array(predictions)
            
            # Create SHAP explainer
            explainer = shap.Explainer(model_wrapper, self.tokenizer)
            
            # Get SHAP values
            shap_values = explainer([text], max_evals=100)
            
            # Display top features
            pred_class = np.argmax(model_wrapper([text])[0])
            if hasattr(shap_values, 'values') and len(shap_values.values.shape) > 2:
                values = shap_values.values[0, :, pred_class]
            else:
                values = shap_values.values[0] if hasattr(shap_values, 'values') else shap_values[0]
            
            tokens = self.tokenizer.tokenize(text)[:len(values)]
            
            # Show top contributing tokens
            token_scores = list(zip(tokens, values))
            token_scores.sort(key=lambda x: abs(x[1]), reverse=True)
            
            print("Top contributing tokens:")
            for token, score in token_scores[:8]:
                if token not in ['[CLS]', '[SEP]', '[PAD]']:
                    print(f"  {token}: {score:.4f}")
            
            # Try to show visualization if possible
            try:
                shap.plots.text(shap_values[0])
            except:
                print("  (Visual plot not available in this environment)")
                
        except Exception as e:
            print(f"⚠️ SHAP analysis failed: {e}")
            print("Falling back to simple gradient analysis...")
            self.run_simple_gradient_analysis(text)
    
    def run_simple_gradient_analysis(self, text):
        """Simple gradient-based token importance as fallback"""
        try:
            inputs = self.tokenizer(text, return_tensors="pt", truncation=True, 
                                  padding=True, max_length=128)
            tokens = self.tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
            
            # Enable gradients
            inputs['input_ids'].requires_grad_(True)
            outputs = self.model(**inputs)
            
            # Get target prediction
            target_class = torch.argmax(outputs.logits, dim=-1)
            target_prob = torch.softmax(outputs.logits, dim=-1)[0, target_class]
            
            # Backward pass
            target_prob.backward()
            gradients = inputs['input_ids'].grad
            
            # Get importance scores
            importance_scores = torch.abs(gradients[0]).detach().numpy()
            
            # Show results
            token_importance = list(zip(tokens, importance_scores))
            token_importance.sort(key=lambda x: x[1], reverse=True)
            
            print("Token importance (gradient-based):")
            for token, score in token_importance[:8]:
                if token not in ['[CLS]', '[SEP]', '[PAD]']:
                    print(f"  {token}: {score:.4f}")
                    
        except Exception as e:
            print(f"⚠️ Gradient analysis also failed: {e}")
    
    def run_lime_analysis(self, text):
        print("\n🔍 LIME Analysis:")
        try:
            def predict_proba_fn(texts):
                preds = []
                for t in texts:
                    if not t.strip():  # Handle empty strings
                        preds.append([0.33, 0.33, 0.34])  # Default uniform distribution
                        continue
                        
                    inputs = self.tokenizer(t, return_tensors="pt", truncation=True, padding=True, max_length=128)
                    with torch.no_grad():
                        outputs = self.model(**inputs)
                        probs = torch.softmax(outputs.logits, dim=-1).numpy()[0]
                    preds.append(probs)
                return np.array(preds)
            
            explainer = LimeTextExplainer(
                class_names=list(self.label_encoder.classes_),
                mode='classification'
            )
            
            explanation = explainer.explain_instance(
                text, 
                predict_proba_fn, 
                num_features=8,
                num_samples=100
            )
            
            print("Top features:")
            for feature, score in explanation.as_list():
                print(f"  {feature}: {score:.4f}")
                
        except Exception as e:
            print(f"⚠️ LIME analysis failed: {e}")
            print("Falling back to simple analysis...")
    
    def run_attention_analysis(self, text):
        """Attention head visualization"""
        print("\n👁️ Attention Analysis:")
        try:
            inputs = self.tokenizer(text, return_tensors="pt", truncation=True, 
                                  padding=True, max_length=128)
            
            # Get model outputs with attention
            with torch.no_grad():
                outputs = self.model(**inputs, output_attentions=True)
            
            # Extract attention weights
            attention = outputs.attentions  # List of attention weights for each layer
            tokens = self.tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
            
            # Average attention across heads and layers for simplicity
            if attention and len(attention) > 0:
                # Use last layer attention
                last_layer_attention = attention[-1][0]  # [num_heads, seq_len, seq_len]
                
                # Average across heads
                avg_attention = last_layer_attention.mean(dim=0)  # [seq_len, seq_len]
                
                # Get attention to CLS token (classification)
                cls_attention = avg_attention[0, 1:-1].cpu().numpy()  # Skip CLS and SEP
                
                # Show top attended tokens
                token_attention = list(zip(tokens[1:-1], cls_attention))  # Skip CLS and SEP
                token_attention.sort(key=lambda x: x[1], reverse=True)
                
                print("Most attended tokens:")
                for token, score in token_attention[:8]:
                    if token not in ['[PAD]']:
                        print(f"  {token}: {score:.4f}")
                
                # Try to show BertViz visualization
                try:
                    from bertviz import head_view
                    # This works best in Jupyter with proper display
                    print("  (For interactive attention visualization, use bertviz.head_view)")
                except ImportError:
                    print("  (bertviz not available for interactive visualization)")
            else:
                print("  No attention weights available from model")
                
        except Exception as e:
            print(f"⚠️ Attention analysis failed: {e}")
    
    def run_gradcam_analysis(self, text):
        """GradCAM analysis"""
        print("\n🌡️ GradCAM Analysis:")
        try:
            inputs = self.tokenizer(text, return_tensors="pt", truncation=True, 
                                  padding=True, max_length=128)
            
            # Get model prediction
            outputs = self.model(**inputs)
            pred_class = torch.argmax(outputs.logits, dim=-1).item()
            
            # Use Captum for GradCAM
            from captum.attr import LayerGradCam
            
            # Create GradCAM for the last transformer layer
            layer_gradcam = LayerGradCam(self.model, self.model.bert.encoder.layer[-1])
            
            # Get attributions
            attributions = layer_gradcam.attribute(
                inputs['input_ids'],
                target=pred_class,
                additional_forward_args=(inputs['attention_mask'],)
            )
            
            # Average across dimensions to get token-level importance
            if len(attributions.shape) > 2:
                token_importance = attributions.mean(dim=-1).squeeze().detach().numpy()
            else:
                token_importance = attributions.squeeze().detach().numpy()
            
            tokens = self.tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
            
            # Show top important tokens
            token_scores = list(zip(tokens, token_importance))
            token_scores.sort(key=lambda x: abs(x[1]), reverse=True)
            
            print("Most important regions (GradCAM):")
            for token, score in token_scores[:8]:
                if token not in ['[CLS]', '[SEP]', '[PAD]']:
                    print(f"  {token}: {score:.4f}")
                    
        except Exception as e:
            print(f"⚠️ GradCAM analysis failed: {e}")
            print("  (This method requires specific model architecture compatibility)")
    
    def display(self):
        display(self.dashboard)

# Create and display dashboard
print("🔧 Creating explainability dashboard...")
dashboard = ExplainabilityDashboard(model, tokenizer, test_texts, test_labels, predictions, label_encoder)
dashboard.display()

🔧 Creating explainability dashboard...


## 5. 🔍 Quick Misclassification Analysis

Simple analysis to identify patterns for fine-tuning in the next notebook.

In [6]:
# Quick misclassification analysis
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import json

# Get misclassified examples
misclassified_mask = predictions != test_labels
misclassified_texts = [test_texts[i] for i in range(len(test_texts)) if misclassified_mask[i]]
misclassified_true = test_labels[misclassified_mask]
misclassified_pred = predictions[misclassified_mask]

print(f"📊 Total misclassifications: {len(misclassified_texts)}")
print(f"📈 Error rate: {len(misclassified_texts)/len(test_texts)*100:.1f}%")

# 1. Confusion patterns
print("\n🔄 Top Confusion Patterns:")
confusion_data = defaultdict(int)
for true_idx, pred_idx in zip(misclassified_true, misclassified_pred):
    true_label = label_encoder.inverse_transform([true_idx])[0]
    pred_label = label_encoder.inverse_transform([pred_idx])[0]
    confusion_data[(true_label, pred_label)] += 1

for (true_label, pred_label), count in sorted(confusion_data.items(), key=lambda x: x[1], reverse=True)[:3]:
    percentage = count / len(misclassified_texts) * 100
    print(f"  {true_label} → {pred_label}: {count} cases ({percentage:.1f}%)")

# 2. Problematic keywords
print("\n🔍 Problematic Keywords:")
correctly_classified_texts = [test_texts[i] for i in range(len(test_texts)) if not misclassified_mask[i]]

vectorizer = TfidfVectorizer(max_features=200, stop_words='english', ngram_range=(1, 2))
all_texts = misclassified_texts + correctly_classified_texts[:len(misclassified_texts)]
vectorizer.fit(all_texts)

misc_tfidf = vectorizer.transform(misclassified_texts).mean(axis=0).A1
correct_tfidf = vectorizer.transform(correctly_classified_texts[:len(misclassified_texts)]).mean(axis=0).A1

feature_names = vectorizer.get_feature_names_out()
score_diff = misc_tfidf - correct_tfidf
top_indices = score_diff.argsort()[-10:][::-1]

problematic_keywords = [(feature_names[i], score_diff[i]) for i in top_indices if score_diff[i] > 0.001]
for keyword, score in problematic_keywords[:5]:
    print(f"  {keyword}: {score:.4f}")

# Save results for fine-tuning notebook
os.makedirs('analysis_results', exist_ok=True)
results = {
    'confusion_patterns': dict(confusion_data),
    'problematic_keywords': problematic_keywords,
    'total_errors': len(misclassified_texts),
    'error_rate': len(misclassified_texts)/len(test_texts)*100
}

with open('analysis_results/misclassification_analysis.json', 'w') as f:
    json.dump(results, f, indent=2)

print(f"\n💾 Results saved to: analysis_results/misclassification_analysis.json")
print(f"📋 Ready for fine-tuning in next notebook!")

📊 Total misclassifications: 209
📈 Error rate: 20.9%

🔄 Top Confusion Patterns:
  neutral → positive: 73 cases (34.9%)
  positive → neutral: 59 cases (28.2%)
  neutral → negative: 35 cases (16.7%)

🔍 Problematic Keywords:
  solutions: 0.0169
  new: 0.0160
  mln: 0.0147
  pct: 0.0134
  compared: 0.0120
  solutions: 0.0169
  new: 0.0160
  mln: 0.0147
  pct: 0.0134
  compared: 0.0120


TypeError: keys must be str, int, float, bool or None, not tuple

## 6. 📋 Summary

### ✅ Completed:
- **Interactive Dashboard**: SHAP and LIME explanations for any text
- **Mistake Analysis**: Analyze specific model errors
- **Misclassification Patterns**: Key insights for fine-tuning

### 📊 Key Findings:
- Error rate: ~20% on test data
- Main confusion patterns identified
- Problematic keywords extracted

### 🔜 Next Steps:
Results saved to `analysis_results/` for **Notebook 6: Fine-tuning with Pruning Methods**